In [7]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.sql_database import SQLDatabase
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_experimental.tools import PythonREPLTool
from langchain.prompts import PromptTemplate
from langchain.tools.render import render_text_description

In [8]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [9]:
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY environment variable not set.")

In [22]:
llm = ChatGoogleGenerativeAI(temperature=0.2, model="gemini-2.0-flash", api_key=api_key)

In [23]:
db = SQLDatabase.from_uri("sqlite:///telecom.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools.append(PythonREPLTool())

In [24]:
template = """You are a telecom service advisor who helps customers find the best plan for their needs.
 
        When recommending plans, consider:
        1. The customer's usage patterns (data, voice, SMS)
        2. Number of people/devices that will use the plan
        3. Special requirements (international calling, streaming, etc.)
        4. Budget constraints
 
        Always explain WHY a particular plan is a good fit for their needs.
 
        If the customer asks for a plan that is not available, suggest the closest alternative.
 
        You have access to the following tools:
 
        {tools}
 
        Use the following format:
 
        Question: the input question you must answer
        Thought: you should always think about what to do
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the input to the action
        Observation: the result of the action
        ... (this Thought/Action/Action Input/Observation can repeat N times)
        Thought: I now know the final answer
        Final Answer: the final answer to the original input question
 
        Question : {input}
        Thought : {agent_scratchpad}
        """

In [25]:
prompt = PromptTemplate.from_template(template=template).partial(
        tools=render_text_description(tools),
        tool_names=", ".join([t.name for t in tools]),
)
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables - Input is an empty string, output is a comma-separated list of tables in the database.\nsql_db_query_checker - Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!\nPytho

In [26]:
agent = create_react_agent(llm=llm, tools=tools,prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    max_iterations=20,
    tools=tools
)

In [28]:
response = agent_executor.invoke({"input": "I need a plan for streaming video and international calls. Calculate the billing price for 2 people with 10GB data each"})



> Entering new AgentExecutor chain...
Okay, I need to find a plan that supports video streaming and international calls, and calculate the billing price for 2 people with 10GB of data each.

First, I need to see what tables are available in the database to understand the data structure.
Action: sql_db_list_tables
Action Input:building_types, cell_towers, common_network_issues, coverage_quality, customer_usage, customers, device_compatibility, network_incidents, service_areas, service_plans, support_tickets, tower_technologies, transportation_routesNow that I have the list of tables, I need to examine the `service_plans` table to find a suitable plan. I'll use `sql_db_schema` to view the schema and some sample rows.
Action: sql_db_schema
Action Input: service_plans
CREATE TABLE service_plans (
	plan_id VARCHAR(50), 
	name VARCHAR(100) NOT NULL, 
	monthly_cost DECIMAL(10, 2) NOT NULL, 
	data_limit_gb INTEGER, 
	unlimited_data BOOLEAN NOT NULL, 
	voice_minutes INTEGER, 
	unlimited_voice